## Creating a Deadlock in Ray

We will build an example in Ray that pushes and pulls data betweeen two actors. This is designed to mimic MPI messaging, which is an awkward way to use Ray.

In [1]:
import ray
import time 

@ray.remote
class PairedSendReceive(object):
    """
    Paired send and receive. This mimics the deadlock breaking behavior of MPI.
    """
    msg = ""

    def __init__(self):
        None

    def set_message(self, msg):
        """set the message to be sent"""
        self.msg = msg

    def push_send(self, ooid):
        """initiate message from the sender"""
        return ray.get(ooid.recv.remote(self.msg))

    def recv(self, msg):
        """remote function called by sender"""
        return msg

    def pull_recv(self, ooid):
        """initiate message from the receiver"""
        # sleep to make sure both actors are starte
        time.sleep(1)
        return ray.get(ooid.send.remote())

    def send(self):
        """remote function called by receiver"""
        return self.msg

ray.init(num_cpus=4, ignore_reinit_error=True)

2023-11-28 21:30:50,930	WARNING services.py:1889 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=3.68gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-11-28 21:30:51,049	INFO worker.py:1642 -- Started a local Ray instance.


Python version:,3.10.9
Ray version:,2.7.1


#### Sending (push) messages

In [2]:
# Create Send/Recv objects
sr0 = PairedSendReceive.remote()
sr1 = PairedSendReceive.remote()

# set messages
roid0 = sr0.set_message.remote(f"push_send test from {sr0}")
roid1 = sr1.set_message.remote(f"push_send test from {sr1}")
ray.get(roid0)
ray.get(roid1)

# synchronized (one after the other) push
roid0 = sr0.push_send.remote(sr1)
print(ray.get(roid0))
roid1 = sr1.push_send.remote(sr0)
print(ray.get(roid1))

push_send test from Actor(PairedSendReceive, d669eabfff5a9d9d3494725501000000)
push_send test from Actor(PairedSendReceive, ee4f1ea36cdc9c7f001e477401000000)


#### Receiving (pull) messages

In [3]:
# set new messages
roid0 = sr0.set_message.remote(f"pull_recv test from {sr0}")
roid1 = sr1.set_message.remote(f"pull_recv test from {sr1}")
ray.get(roid0)
ray.get(roid1)

# synchronized (one after the other) pull
roid0 = sr0.pull_recv.remote(sr1)
print(ray.get(roid0))
roid1 = sr1.pull_recv.remote(sr0)
print(ray.get(roid1))


pull_recv test from Actor(PairedSendReceive, ee4f1ea36cdc9c7f001e477401000000)
pull_recv test from Actor(PairedSendReceive, d669eabfff5a9d9d3494725501000000)


OK, so far so good.  We can send and receive messages by:
  * send: create a message call remote to receive 
  * receive: call remote to get its message

#### Deadlock

Both of these examples are serial with the `get` function creating a synchronization point between the two messages.

What happens with concurrent messages?

In [4]:
# synchronized (one after the other) pull
#roid0 = sr0.pull_recv.remote(sr1)
#roid1 = sr1.pull_recv.remote(sr0)
#ray.get([roid0,roid1])

OK, that's a deadlock. Comment this out and then restart the kernel. Why is it a deadlock?

Because each Ray actor only has a single execution context.  So, it meets the deadlock criteria.
  * two resources -- execution context of actor 0 and actor 1
  * two holders -- `pull_recv` function on each actor
  * two waiters -- remotely invoked `send` function
  
We can see it meets all the criteria.
  * circular dependency
  * hold and wait
  * no preemption
  * mutual exclusion
  
This deadlock persists indefinitely until we restart the kernel (breaking the deadlock with preemption).

#### Resolving the Deadlock

The idea here is to pair senders and receivers so that one is sending while the other is receiving.
This ends up invoking both the send and the receive from the same node.

In [5]:
# set new messages
roid0 = sr0.set_message.remote(f"paired push_send test from {sr0}")
roid1 = sr1.set_message.remote(f"paired pull_recv test from {sr1}")
ray.get(roid0)
ray.get(roid1)

# concurrent paired send and receive
roid0 = sr0.push_send.remote(sr1)
roid1 = sr0.pull_recv.remote(sr1)
print(ray.get(roid0))
print(ray.get(roid1))

paired push_send test from Actor(PairedSendReceive, d669eabfff5a9d9d3494725501000000)
paired pull_recv test from Actor(PairedSendReceive, ee4f1ea36cdc9c7f001e477401000000)


Doing this with only two actors makes sequential. It becomes more interesting (and actually concurrent) when we do it with more actors. Let's try 4 actors organized in a ring.

In [6]:
# Create Send/Recv objects
sr0 = PairedSendReceive.remote()
sr1 = PairedSendReceive.remote()
sr2 = PairedSendReceive.remote()
sr3 = PairedSendReceive.remote()

# set new messages
roid0 = sr0.set_message.remote(f"message actor 0: {sr0}")
roid1 = sr1.set_message.remote(f"message actor 1: {sr1}")
roid2 = sr2.set_message.remote(f"message actor 2: {sr2}")
roid3 = sr3.set_message.remote(f"message actor 3: {sr3}")
ray.get([roid0,roid1,roid2,roid3])

# concurrent paired send and receive
roid0 = sr0.push_send.remote(sr1)
roid1 = sr0.pull_recv.remote(sr1)
roid2 = sr0.push_send.remote(sr3)
roid3 = sr0.pull_recv.remote(sr3)
roid4 = sr2.push_send.remote(sr1)
roid5 = sr2.pull_recv.remote(sr1)
roid6 = sr2.push_send.remote(sr3)
roid7 = sr2.pull_recv.remote(sr3)
print(ray.get(roid0))
print(ray.get(roid1))
print(ray.get(roid2))
print(ray.get(roid3))
print(ray.get(roid4))
print(ray.get(roid5))
print(ray.get(roid6))
print(ray.get(roid7))

message actor 0: Actor(PairedSendReceive, 476ba00c4e65d4fd05ca0d6201000000)
message actor 1: Actor(PairedSendReceive, d90add7322b6ffb02d224b1e01000000)
message actor 0: Actor(PairedSendReceive, 476ba00c4e65d4fd05ca0d6201000000)
message actor 3: Actor(PairedSendReceive, d864f251c4a252e844ac777401000000)
message actor 2: Actor(PairedSendReceive, 12d8d35ed22ae595688f015f01000000)
message actor 1: Actor(PairedSendReceive, d90add7322b6ffb02d224b1e01000000)
message actor 2: Actor(PairedSendReceive, 12d8d35ed22ae595688f015f01000000)
message actor 3: Actor(PairedSendReceive, d864f251c4a252e844ac777401000000)


This ends up being an hard way to think about it.  It is easier to conceive of this as even/odd nodes that are sending and receiving. And that sending and receiving occurs in pairs.

<img src=../images/pairedsr.png>

In [7]:
# concurrent paired send and receive

# send from even to odd
roid0 = sr0.push_send.remote(sr1)
roid1 = sr0.push_send.remote(sr3)
roid2 = sr2.push_send.remote(sr3)
roid3 = sr2.push_send.remote(sr1)
print(ray.get(roid0))
print(ray.get(roid1))
print(ray.get(roid2))
print(ray.get(roid3))

# send from odd to even
roid0 = sr1.push_send.remote(sr0)
roid1 = sr1.push_send.remote(sr2)
roid2 = sr3.push_send.remote(sr0)
roid3 = sr3.push_send.remote(sr2)
print(ray.get(roid0))
print(ray.get(roid1))
print(ray.get(roid2))
print(ray.get(roid3))

message actor 0: Actor(PairedSendReceive, 476ba00c4e65d4fd05ca0d6201000000)
message actor 0: Actor(PairedSendReceive, 476ba00c4e65d4fd05ca0d6201000000)
message actor 2: Actor(PairedSendReceive, 12d8d35ed22ae595688f015f01000000)
message actor 2: Actor(PairedSendReceive, 12d8d35ed22ae595688f015f01000000)
message actor 1: Actor(PairedSendReceive, d90add7322b6ffb02d224b1e01000000)
message actor 1: Actor(PairedSendReceive, d90add7322b6ffb02d224b1e01000000)
message actor 3: Actor(PairedSendReceive, d864f251c4a252e844ac777401000000)
message actor 3: Actor(PairedSendReceive, d864f251c4a252e844ac777401000000)


#### Breaking Deadlock with Queuing

If we buffer messages in queues, we can make sending  asynchronous.  The sending process now buffers a message in memory and the receiver retrieves it from memory. Send doesn't wait for receive.

In [8]:
@ray.remote
class QueuedSendReceive(object):
    """
    Send and receive through queues.
    Queues make the send call asynchronous.

    # RB note the queue breaks the send/send deadlock
    # RB note how many queues are needed?  why inferior to push model
    """
    msg = ""

    def __init__(self, squeue, rqueue):
        """Create an inbound and outbound queue for each actor."""
        self.recvQ = rqueue
        self.sendQ = squeue

    def set_message(self, msg):
        """set the message to be sent"""
        self.msg = msg

    def _qsend(self):
        """Helper: enqueue a message"""
        ### TODO
        self.sendQ.put(self.msg)

    def _qreceive(self):
        """Helper: dequeue a message"""
        ### TODO
        msg = self.recvQ.get()
        return msg

    def rs_exchange(self):
        """receive first then send"""
        ### TODO
        msg = self._qreceive()
        self._qsend()
        return msg
    
    def sr_exchange(self):
        """send first then receive"""
        ### TODO
        self._qsend()
        return self._qreceive()

In [10]:
from ray.util.queue import Queue

# script to drive parallel program
ray.init(num_cpus=4, ignore_reinit_error=True)

# create messaging queues
q0to1 = Queue(maxsize=100)
q1to0 = Queue(maxsize=100)

# objects with paired queues
sr0 = QueuedSendReceive.remote(q0to1,q1to0)
sr1 = QueuedSendReceive.remote(q1to0,q0to1)

# set messages
roid0 = sr0.set_message.remote(f"Message from {sr0}")
roid1 = sr1.set_message.remote(f"Message from {sr1}")
ray.get(roid0)
ray.get(roid1)

print("Send then receive. No deadlock")
oid0 = sr0.sr_exchange.remote()
oid1 = sr1.sr_exchange.remote()
print(ray.get(oid0))
print(ray.get(oid1))

2023-11-28 21:33:56,769	INFO worker.py:1476 -- Calling ray.init() again after it has already been called.


Send then receive. No deadlock
Message from Actor(QueuedSendReceive, b228d5398913b0a9e61fe13101000000)
Message from Actor(QueuedSendReceive, ebd32ec797328f681f4e09a401000000)


### Queuing Tradeoffs

Buffering is a powerful technique for enabling concurrency. It can time shift the delivery of data from sending and receiving processes. It plays a similar role as caching with processes trying to write to devices. 

Sophisticated implementations allow for one physical queue per actor/process by tagging messages with sender data and allowing receivers to receive messages from specific senders.

The problem with queueing is one of scale. When queues run out of memory or storage, the send process becomes synchronous. It must wait for queue space before returning. 

This means that deadlock can occur even in queueing systems, particularly when they scale to many parties or heavy workloads.

**Conclusion**: It is important to use deadlock-free messaging disciplines even with buffering.